In [1]:
import torch
import numpy as np

import os
os.chdir('..')
from torch_geometric.loader import DataLoader
from gnn_library.util import train, save, load
from evaluate import evaluate_model, pp_output
import instance_generator as ig
import torch_converter as tc
import evaluate as ev
import osmnx as ox
from util import Dataset
from gnn_library.OBM_threshold_greedy import OBM_Threshold_Greedy
import experiment_parameters as params

%load_ext autoreload
%autoreload 2

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("PyTorch has version {}".format(torch.__version__))
print('Using device:', device)

PyTorch has version 1.12.0+cu102
Using device: cuda:0


In [3]:
# Hyperparam optimized for 10,6

args = {
    'processor':         'GENConv',
    'head':              'regression',     
    'num_layers':        3,
    'num_mlp_layers':    3,
    'aggr':              'max',
    'batch_size':        8,
    'node_feature_dim':  5,
    'edge_feature_dim':  1,
    'graph_feature_dim': 2,
    'hidden_dim':        32,
    'output_dim':        1,
    'dropout':           0.0306,
    'epochs':            64,
    'opt':               'adagrad',
    'opt_scheduler':     'none',
    'opt_restart':       0,
    'weight_decay':      5e-3,
    'lr':                0.0121,
    'device':            device,
    'noise':             0 # Set to 0 to not train on noisy features
}

## Set up the threshold greedy value

In [ ]:
from tqdm import tqdm
import util
seed = 0
rng = np.random.default_rng(0)


THRESHOLDS = np.linspace(0, 1, 101)

thresholded_greedy_models = {threshold: OBM_Threshold_Greedy(threshold) for threshold in THRESHOLDS}

# Node configs and graph configs on which threshold greedy is optimized
node_configs = params.THRESHOLD_GREEDY_NODE_REGIMES
graph_configs = params.ALL_TESTING_GRAPH_CONFIGS
train_num = 20

train_instances = [
        ig.sample_instances(node_config[0], node_config[1], train_num, rng, args, **graph_config)
        for graph_config in graph_configs
        for node_config in node_configs
]

# flat map the train instances
train_instances = [instance for instances in train_instances for instance in instances]

greedy_ratios = {}
for threshold, model in tqdm(thresholded_greedy_models.items()): 
    ratio, _ = ev.evaluate_model(
        meta_model=None,
        meta_model_type=None,
        base_models=[model],
        instances=train_instances,
        batch_size=50,
        rng=rng,
        num_realizations=5
    )
    greedy_ratios[threshold] = np.mean(ratio['learned'])
    

max_threshold = max(greedy_ratios, key = greedy_ratios.get)
print(f"Best threshold value: {max_threshold} achieves CR: {greedy_ratios[max_threshold]}")
THRESHOLD_MODEL = thresholded_greedy_models[max_threshold]

In [ ]:
print(greedy_ratios)